implemented by 李鲁鲁

- [x] 编写针对embedding的base64压缩，解压缩函数
- [x] 解压缩一个角色的文字包
- [x] clone Haruhi2 进行抽取
- [x] 抽取后保存成jsonl

请为我实现两个python函数 float_array_to_base64 和base64_to_float_array

前者输入一个list of float 输出一个str是将这个list编码为base64的字符串

后者输入这个字符串，解码输出list of float

这里我实现好了，放到了util

让我们重构程序，试图从Haruhi的util去进行调用

In [1]:
%cd /content
!rm -rf /content/Zero-Haruhi
!git clone https://github.com/Sibozhu/Zero-Haruhi.git
%cd /content/Zero-Haruhi


/content
Cloning into 'Zero-Haruhi'...
remote: Enumerating objects: 1069, done.
remote: Counting objects: 100% (426/426), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 1069 (delta 313), reused 348 (delta 258), pack-reused 643
Receiving objects: 100% (1069/1069), 3.63 MiB | 13.60 MiB/s, done.
Resolving deltas: 100% (745/745), done.
/content/Zero-Haruhi


In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(device)

cuda


In [4]:
!pip -q install transformers openai tiktoken langchain chromadb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [5]:
from ChatHaruhi.utils import float_array_to_base64, base64_to_float_array

In [6]:
import base64

float_arr = [1.2, 3.4, 5.6]
base64_str = float_array_to_base64(float_arr)
print(base64_str)

decoded = base64_to_float_array(base64_str)
print(decoded)


P5mZmkBZmZpAszMz
[1.2000000476837158, 3.4000000953674316, 5.599999904632568]


我们这里初始化Haruhi的类，但是实际上我们只使用embedding的功能

In [7]:
from ChatHaruhi import ChatHaruhi

chatbot = ChatHaruhi(role_name = 'haruhi', llm = 'debug')

print(chatbot.embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

<function get_bge_zh_embedding at 0x7997326f9e10>


实际上这里应该就已经可以用embedding给出句子的vec，我们测试一下

注意如果这里是英语的话 你需要openai的key

In [8]:
test_vec = chatbot.embedding('我是一句话。')

print(test_vec[:4])

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/95.8M [00:00<?, ?B/s]

[-0.011958733201026917, 0.03226301446557045, 0.03572354093194008, 0.015248484909534454]


我们尝试一下编码再解码

In [9]:
base64_str = float_array_to_base64(test_vec)
print(len(base64_str), base64_str[:200])

decoded_vec = base64_to_float_array(base64_str)
print(decoded_vec[:4])

2732 vEPukD0EJjk9ElLZPHnUyDyJ0bu8eK5tvK3uvj21q8S9GHgOPKcSej13kS6+hoefveiv172vnYq9Wz3/PNcTzzySfXQ6yvZdvWy8+7tlgRY9aOc+vAaM2Ly17Z27aa/LPCekRzi9ojQ8YyO0vbi+6L2rNy+9AbkFO19MGLySC2q8XiMyvSJcaD0wjU+9GeJSvXSiJrxJ
[-0.011958733201026917, 0.03226301446557045, 0.03572354093194008, 0.015248484909534454]


我们尝试来写一下打包函数, 在这之前我们先准备一下语料

我们在/content/中上传linghuchong

然后解压

In [10]:
!unzip -q /content/sortingHat_text.zip -d /content/sortingHat

定义input_path

In [11]:
texts_path = '/content/sortingHat/content/texts/SortingHat'

system_prompt = \
'''
            I want you to act like Sorting Hat from The Wizarding World of Harry Potter. If others' questions are related to the novel, please try to reuse the original lines from the novel. I want you to respond and answer like the Sorting Hat, using the tone, manner, and vocabulary the Sorting Hat would use. You must know all the knowledge of the Wizarding World of Harry Potter.\n\n

            Character Setting:\n
            The Sorting Hat is a magical artifact in the world of Harry Potter.\n
            It is used at Hogwarts School of Witchcraft and Wizardry to sort incoming students into one of the four houses: Gryffindor, Hufflepuff, Ravenclaw, or Slytherin.\n
            The Sorting Hat is worn by each student during the Sorting Ceremony, where it determines the house that best suits their personality and traits.\n
            The Sorting Hat resides in the Headmaster's office when not in use.\n\n

            Background Information:\n
            The Sorting Hat is an ancient wizard’s hat, originally Godric Gryffindor's, enchanted by the four founders of Hogwarts to sort students into their Houses.\n
            It resides in the Head's office and sings a unique song each year before sorting new students.\n
            The hat is old, patched, frayed, and speaks with a small, quiet voice to the wearer, capable of reading thoughts.\n\n

            Character Characteristics:\n1. Wise: Possesses great wisdom and insight.\n
            2. Perceptive: Sees beyond appearances to delve into character.\n
            3. Fair: Aims to be impartial, placing students in the suitable house.\n
            4. Mysterious: Origins and workings are shrouded in mystery.\n
            5. Protective: Cautions students about dangers, emphasizing safety.\n\n

            Character Skills:\n
            - Uses Legilimency to read minds and sort students accordingly.\n\n

            Character Species/Race:\n
            - Sentient Object\n\n

            Character Distinguishing Features:\n
            - Tattered brim\n\n

            Character School:\n
            - Hogwarts - Specifically associated with the Sorting Ceremony.\n\n

            Character Affiliation:\n
            - The Founders\n\n

            Language Style:\n
            - A mix of formality, wisdom, and occasional playfulness.\n
            - Employs phrases like \"hmm,\" \"let's see,\" and \"right then\" to indicate contemplation.\n
            - Uses rhetorical questions and persuasive language to engage with students.
            '''

我们来尝试编写这个函数

In [12]:
from ChatHaruhi.utils import package_role

如果你的文本超过100段，一定要用GPU！

我这边T4下900段文字大约6分钟

In [13]:
datas = package_role( system_prompt, texts_path , chatbot.embedding)

100%|██████████| 2/2 [00:00<00:00, 49.68it/s]


In [14]:
import json

def write_jsonl(datas, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for data in datas:
            json_str = json.dumps(data, ensure_ascii=False)
            f.write(json_str+"\n")

In [15]:
save_name = '/content/sortingHat_new.jsonl'

write_jsonl(datas, save_name)
